In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys #bould use to neviagete more interactively
import time

In [ ]:
link='https://data.gov.in/'
dataGov=webdriver.Chrome()
dataGov.get(link)

In [ ]:
## --------------- although can do without logIn(comment this cell) but I wanted it to be authentic -------- ##
# login
time.sleep(5)
login=dataGov.find_element_by_xpath('//*[@id="block-common-utility-comutil-user-login-new-block"]/div/a[1]')
login.click()

# user input
number=input('your dataGov registered mobile no: ')
passwd=input('your corresponding password: ')
time.sleep(5)
# login input
mobile=dataGov.find_element_by_xpath('//*[@id="edit-name"]')
mobile.send_keys(number)
pwd=dataGov.find_element_by_xpath('//*[@id="edit-pass"]')
pwd.send_keys(passwd)
pwd.submit()

In [ ]:
time.sleep(5)
apibtn=dataGov.find_element_by_xpath('//*[@id="block-views-home-manager-index-block-2"]/div/div/div/div/ul/li/div[2]/span[2]/a')
apibtn.click()
time.sleep(5)

In [ ]:
def total_pages():
    # got to the last element
    time.sleep(2)
    last=dataGov.find_element_by_xpath('//*[@id="apis_listings"]/div[2]/nav/ul/li[7]/a')
    last.click()
    # let it load an then get total pages count 
    time.sleep(5)
    last_digit=dataGov.find_element_by_xpath('//*[@id="apis_listings"]/div[2]/nav/ul/li[5]/a')
    last_digit=int(last_digit.text)
    # go back to the first page
    time.sleep(2)
    first=dataGov.find_element_by_xpath('//*[@id="apis_listings"]/div[2]/nav/ul/li[1]/a')
    first.click()
    time.sleep(5)
    return last_digit

no_of_pages=total_pages()

In [ ]:
print('total number of API pages -----',no_of_pages)

In [ ]:
api_link=list()
api_title=list()
api_departments=list()
api_sectors=list()
date_created=list()
last_update=list()

requested_pages=int(input('how many pages to scrap -->'))
counter=1
curr_soup=bs(dataGov.page_source.encode('utf-8'))
while((counter <= requested_pages) and (counter <= no_of_pages)):
    print('now processing page no .......',counter)
    time.sleep(5)
    for data in curr_soup.find_all('div',{'class':'apifrom_dataset'}):
        # link and title
        api_link.append(data.a.get('href'))
        api_title.append(data.a.h3.string)
        # departments
        # needs bit cleaning
        api_departments.append(data.p.get_text())
        #sectors
        #needs cleaning
        api_sectors.append(data.div.get_text())
        # if-else to ensure programme does not get stuck due to some missing data
        # if-else can be added for above but IG they will be there; if there should be any entry
        if data.find('div',{'class':'created_date'})!=None:
            date_created.append(data.find('div',{'class':'created_date'}).get_text())
        else:
            date_created.append('notFound')
        if data.find('div',{'class':'updated_date'})!=None:
            last_update.append(data.find('div',{'class':'updated_date'}).get_text())
        else:
            last_update.append('notFound')
    counter+=1
    if counter <= no_of_pages: # just to avoid last page not finding next element
        next=dataGov.find_element_by_partial_link_text('Next >')
        next.click()
        time.sleep(5)
        curr_soup=bs(dataGov.page_source.encode('utf-8'))
dataGov.quit()